In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize

# Parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

# Paths
model_path = 'C:/Users/lenovo/Documents/GitHub/Mechatronics/Codes/crop_row_detection_model.h5'
image_path = 'C:/Users/lenovo/Documents/GitHub/Mechatronics/Codes/GrassSample.png'

# Load the trained model
model = tf.keras.models.load_model(model_path,compile = False)

# Load and preprocess the input image
def preprocess_image(image_path):
    img = imread(image_path)[:,:,:IMG_CHANNELS]  # Load image and select the first 3 channels
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)  # Resize image
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

input_image = preprocess_image(image_path)

# Predict the mask
prediction = model.predict(input_image)[0]
predicted_mask = (prediction > 0.5).astype(np.uint8)

# Visualize the results
def display_results(original_image_path, predicted_mask):
    original_image = imread(original_image_path)[:,:,:IMG_CHANNELS]  # Load original image

    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.title('Original Image')
    plt.imshow(original_image)
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.title('Predicted Mask')
    plt.imshow(predicted_mask.squeeze(), cmap='gray')
    plt.axis('off')

    plt.show()

display_results(image_path, predicted_mask)


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize

# Parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

# Paths
model_path = 'C:/Users/lenovo/Documents/GitHub/Mechatronics/Codes/crop_row_detection_model.h5'
image_path = 'C:/Users/lenovo/Documents/GitHub/Mechatronics/paper Submission/Aerial Image2.png'

# Load the trained model
model = tf.keras.models.load_model(model_path, compile=False)

# Load and preprocess the input image
def preprocess_image(image_path):
    img = imread(image_path)[:,:,:IMG_CHANNELS]  # Load image and select the first 3 channels
    img_height, img_width, _ = img.shape
    
    # Calculate the dimensions to crop the image to be divisible by 128
    crop_height = img_height - (img_height % IMG_HEIGHT)
    crop_width = img_width - (img_width % IMG_WIDTH)
    
    # Crop the image
    cropped_img = img[:crop_height, :crop_width, :]
    
    return cropped_img

input_image = preprocess_image(image_path)

# Split the image into patches
def split_image(img, patch_size):
    patches = []
    img_height, img_width, _ = img.shape
    for y in range(0, img_height, patch_size):
        for x in range(0, img_width, patch_size):
            patch = img[y:y+patch_size, x:x+patch_size]
            patches.append((patch, (x, y)))
    return patches

# Reconstruct the image from patches
def reconstruct_image(patches, img_shape, patch_size):
    reconstructed_img = np.zeros(img_shape, dtype=np.float32)
    count_matrix = np.zeros(img_shape, dtype=np.float32)
    
    for patch, (x, y) in patches:
        h, w = patch.shape[:2]

        # Expand the 2D mask (128x128) to a 3D mask (128x128x3) if necessary
        if len(patch.shape) == 2:
            patch = np.repeat(patch[:, :, np.newaxis], 3, axis=2)

        # Add the patch to the reconstructed image
        reconstructed_img[y:y+h, x:x+w] += patch
        
        # Increment the count matrix (this tracks how many patches contribute to each pixel)
        count_matrix[y:y+h, x:x+w] += 1

    # Normalize by the number of patches contributing to each pixel
    # Add a small value to count_matrix to avoid division by zero
    count_matrix = np.maximum(count_matrix, 1)
    reconstructed_img /= count_matrix

    # Clip values to ensure they are within valid image range
    return np.clip(reconstructed_img, 0, 255).astype(np.uint8)

# Split the image into patches
patches = split_image(input_image, IMG_WIDTH)

# Predict the mask for each patch
predicted_patches = []
for patch, (x, y) in patches:
    patch_resized = resize(patch, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    patch_resized = np.expand_dims(patch_resized, axis=0)
    prediction = model.predict(patch_resized)[0]
    predicted_mask_patch = (prediction > 0.5).astype(np.uint8) * 255  # Convert to 0-255 range
    predicted_patches.append((predicted_mask_patch.squeeze(), (x, y)))

# Reconstruct the predicted mask from patches
reconstructed_mask = reconstruct_image(predicted_patches, input_image.shape, IMG_WIDTH)

# Visualize the results
def display_results(original_image, reconstructed_mask):
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(original_image)
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.title('Reconstructed Mask')
    plt.imshow(reconstructed_mask, cmap='gray')
    plt.axis('off')

    plt.show()

display_results(input_image, reconstructed_mask)

